In [1]:
#%pip install azure-ai-projects

## Example of using three Azure AI agents with AUTOGEN as the orchestrator

In [2]:

from PIL import Image
from pathlib import Path
from IPython.display import display, HTML
import os
import sys
import time


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from azure.ai.agents import AgentsClient 
from azure.ai.agents.models import CodeInterpreterTool, BingGroundingTool
import os
import yaml

from dotenv import load_dotenv
load_dotenv()

from azure.identity import DefaultAzureCredential

In [3]:
with open("model_config.yaml", "r") as f:
        model_config = yaml.safe_load(f)
model_client = AzureOpenAIChatCompletionClient.load_component(model_config)

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


In [5]:
project_client = AgentsClient(
    credential=DefaultAzureCredential(),
    endpoint=endpoint
)

In [6]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# define the web_ai_agent, with BING grounding

In [7]:
async def web_ai_agent(query: str) -> str:
    print()
    print("This is Bing for Azure AI Agent Service .......")
    print()
    bing = BingGroundingTool(connection_id=conn_id)
    # with project_client:
    agent = project_client.create_agent(
            model="gpt-4.1-mini",
            name="my-web-agent",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=query,
    )
    print(f"SMS: {message}")

    # Create and process agent run in thread with tools
    run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    #messages = project_client.messages.list(thread_id=thread.id)
    #print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])

    messages_paged = project_client.messages.list(thread_id=thread.id)
    messages = list(messages_paged)
    if messages:
        print("Messages:" + messages[0]["content"][0]["text"]["value"])
        return messages[0]["content"][0]["text"]["value"]
    else:
        print("No messages found.")
        return ""

        # project_client.close()

    #return messages["data"][0]["content"][0]["text"]["value"]



# Define the blog writer agent with Code Interpreter tool

In [8]:
from IPython.display import FileLink

async def save_blog_agent(blog_content: str) -> str:
    print()
    print("This is Code Interpreter for Azure AI Agent Service .......")

    code_interpreter = CodeInterpreterTool()
    agent = project_client.create_agent(
        model="gpt-4.1-mini",
        name="my-coder-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
    )

    thread = project_client.threads.create()

    message = project_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=f"""
            You are my Python programming assistant. Generate code, save the following blog content, and execute it according to the requirements:

            {blog_content}

            1. Save blog content to blog-Agentic_AI_Blog.md
            2. Give me the download link for this file.
        """,
    )

    run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    messages_paged = project_client.messages.list(thread_id=thread.id)
    messages = list(messages_paged)
    if messages:
        print("Messages:" + messages[0]["content"][0]["text"]["value"])
    else:
        print("No messages found.")

    # Save the file to a notebook-accessible directory
    file_name = f"AI_Blog_{time.strftime('%Y%m%d%H%M%S')}.md"
    target_dir = "./blog"
    os.makedirs(target_dir, exist_ok=True)
    file_path = os.path.join(target_dir, file_name)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(blog_content)
    print(f"File saved: {file_path}")

    # Display a download link in the notebook
    display(FileLink(file_path, result_html_prefix="Download your blog: "))

    project_client.delete_agent(agent.id)
    print("Deleted agent")

    return "Saved"

## Define the three agents

In [9]:
bing_search_agent = AssistantAgent(
    name="bing_search_agent",
    model_client=model_client,
    tools=[web_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)

save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when the blog is saved.
    """
)

write_agent = AssistantAgent(
    name="write_agent",
    model_client=model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)


In [10]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Define the AUTOGEN team of Azure AI agents

In [11]:

reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

## start the team chat

In [ ]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing one blog about machine learning. 
                    Search for the following 4 questions and write a blog based on the search results ,
                    save it, and give me the download this file link
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                    4. The future of Machine Learning
                    
                    
    """)
)  # Stream the messages to the console.

---------- TextMessage (user) ----------


                    I am writing one blog about machine learning. 
                    Search for the following 3 questions and write a blog based on the search results ,
                    save it, and give me the download this file link

                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                    4. The future of Machine Learning


    

This is Bing for Azure AI Agent Service .......

Created agent, ID: asst_JT8zqfh9C24oTVvFh2aMXo30
Created thread, ID: thread_944fPK9woUnDWK73n7Tq5Chd
SMS: {'id': 'msg_vtnxyAp9QbewWEAe0UqTax8y', 'object': 'thread.message', 'created_at': 1750186859, 'assistant_id': None, 'thread_id': 'thread_944fPK9woUnDWK73n7Tq5Chd', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Machine Learning?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finishe

/Users/arturoquiroga/GITHUB/AZURE AI AGENT SERVICE/1_Azure Agent Service/NOTEBOOKS/AUTOGEN/blog/AI_Blog_20250617150156.md

Deleted agent
---------- ToolCallRequestEvent (save_blog_content_agent) ----------
[FunctionCall(id='call_VZk7ME3umCL0ogh6Yy8Uwdra', arguments='{"blog_content":"# Understanding Machine Learning: Past, Present, and Future\\n\\nMachine learning (ML) is revolutionizing the way we live, work, and interact with technology. From powering voice assistants and recommendation engines to enabling autonomous vehicles and medical diagnostics, ML has become a cornerstone of modern innovation. In this blog, we\'ll explore what machine learning is, how it differs from artificial intelligence (AI), delve into its rich history, and look ahead to its promising future.\\n\\n## What is Machine Learning?\\n\\nMachine Learning is a specialized branch of artificial intelligence focused on building systems that learn from data. Unlike traditional software that follows explicitly programmed instructions, ML algorithms analyze patterns from vast amounts of data to make predictions or decisions without being exp

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                    I am writing one blog about machine learning. \n                    Search for the following 3 questions and write a blog based on the search results ,\n                    save it, and give me the download this file link\n\n                    1. What is Machine Learning?\n                    2. The difference between AI and ML\n                    3. The history of Machine Learning\n                    4. The future of Machine Learning\n\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=94), metadata={}, content=[FunctionCall(id='call_nD3YnhgOjKFeSb24d3QphcPl', arguments='{"query": "What is Machine Learning?"}', name='web_ai_agent'), FunctionCall(id='call_lvcaZFlvnMZOZBLtAJLvVJ6U', arguments='{"query": "The difference between AI and ML"}', name='web_ai_agent'), FunctionCall(id='cal

In [ ]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing one blog about agentic AI. 
                    Search for the following 4 questions and write a blog based on the search results ,
                    save it, and give me the download this file link

                    1. What is Agentic AI?
                    2. The difference between AI and Agentic AI
                    3. The history of Agentic AI
                    4. The future of Agentic AI


    """)
)  # Stream the messages to the console.

---------- TextMessage (user) ----------


                    I am writing one blog about agentic AI. 
                    Search for the following 3 questions and write a blog based on the search results ,
                    save it, and give me the download this file link

                    1. What is Agentic AI?
                    2. The difference between AI and Agentic AI
                    3. The history of Agentic AI
                    4. The future of Agentic AI


    

This is Bing for Azure AI Agent Service .......

Created agent, ID: asst_aC4gRXiJsVJ20z6GddElbTnU
Created thread, ID: thread_yGF0E7i0jf6zxQ1qEbPnNb8H
SMS: {'id': 'msg_dDVoObvH7jKA3o7UexgwMdTu', 'object': 'thread.message', 'created_at': 1750186919, 'assistant_id': None, 'thread_id': 'thread_yGF0E7i0jf6zxQ1qEbPnNb8H', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Agentic AI?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: RunStat

/Users/arturoquiroga/GITHUB/AZURE AI AGENT SERVICE/1_Azure Agent Service/NOTEBOOKS/AUTOGEN/blog/AI_Blog_20250617150322.md

Deleted agent
---------- ToolCallRequestEvent (save_blog_content_agent) ----------
[FunctionCall(id='call_T4MjWO8C2D6l2kfxLcinkmj3', arguments='{"blog_content":"# Exploring Agentic AI: Autonomy, History, and the Road Ahead\\n\\nArtificial intelligence (AI) is evolving at an unprecedented pace, and a pivotal frontier in this evolution is **Agentic AI**. While traditional AI technologies have already transformed countless industries, agentic AI is set to redefine autonomy and intelligence in machines by enabling them to independently perceive, decide, and act toward complex goals. This blog will cover what agentic AI is, how it differs from general AI, its historical development, and what the future holds for this fascinating technology.\\n\\n---\\n\\n## What is Agentic AI?\\n\\nAgentic AI refers to AI systems designed to act as autonomous agents \\u0013 systems capable of making independent decisions and performing tasks without continuous human input. Unlike classical AI that may respo

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                    I am writing one blog about agentic AI. \n                    Search for the following 3 questions and write a blog based on the search results ,\n                    save it, and give me the download this file link\n\n                    1. What is Agentic AI?\n                    2. The difference between AI and Agentic AI\n                    3. The history of Agentic AI\n                    4. The future of Agentic AI\n\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=2808, completion_tokens=99), metadata={}, content=[FunctionCall(id='call_AZQT2nokSNFZus86Hx3pvUz8', arguments='{"query": "What is Agentic AI?"}', name='web_ai_agent'), FunctionCall(id='call_8EyDndA4DDJRWujhgYWcWjSv', arguments='{"query": "The difference between AI and Agentic AI"}', name='web_ai_agent'), FunctionCall(id='call_UQ0DVFIceB0

In [14]:


await Console(
    reflection_team.run_stream(task="""

                   I am writing one blog about cybersecurity.
                    Search for the following 4 questions and write a blog based on the search results,
                    save it, and give me the download this file link

                    - What is Cybersecurity?
                    - Differences between cybersecurity and information security
                    - The evolution of cybersecurity practices over time
                    - The future trends and threats in cybersecurity

    """)
)  # Stream the messages to the console.

---------- TextMessage (user) ----------


                   I am writing one blog about cybersecurity.
                    Search for the following 4 questions and write a blog based on the search results,
                    save it, and give me the download this file link

                    - What is Cybersecurity?
                    - Differences between cybersecurity and information security
                    - The evolution of cybersecurity practices over time
                    - The future trends and threats in cybersecurity

    

This is Bing for Azure AI Agent Service .......

Created agent, ID: asst_FAE7fPaSNyOPGE9wA3QQaXTE
Created thread, ID: thread_E4pZH1ppITSUVCnpKDjz3Lum
SMS: {'id': 'msg_uCcHeRYcPkkFUsGDM39pSaPA', 'object': 'thread.message', 'created_at': 1750192577, 'assistant_id': None, 'thread_id': 'thread_E4pZH1ppITSUVCnpKDjz3Lum', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Cybersecurity?', 'annotations': []}}], 'a

/Users/arturoquiroga/GITHUB/AZURE AI AGENT SERVICE/1_Azure Agent Service/NOTEBOOKS/AUTOGEN/blog/AI_Blog_20250617163716.md

Deleted agent
---------- ToolCallRequestEvent (save_blog_content_agent) ----------
[FunctionCall(id='call_F4cXyIlZRD6atnQPyPxsu5Wj', arguments='{"blog_content":"# Understanding Cybersecurity: Past, Present, and Future\\n\\nIn today’s digital landscape, cybersecurity has become a critical component of safeguarding our personal, corporate, and governmental data. With cyber threats constantly evolving, understanding cybersecurity, its development, and emerging challenges is vital for everyone involved. In this blog, we’ll explore what cybersecurity is, how it differs from information security, trace its historical evolution, and take a look into future trends and threats shaping this ever-changing field.\\n\\n---\\n\\n## What is Cybersecurity?\\n\\nCybersecurity is the practice of protecting computer systems, networks, programs, and data from unauthorized digital attacks. These attacks typically aim to access, alter, destroy, or extort sensitive information, or disrupt normal business ope

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                   I am writing one blog about cybersecurity.\n                    Search for the following 4 questions and write a blog based on the search results,\n                    save it, and give me the download this file link\n\n                    - What is Cybersecurity?\n                    - Differences between cybersecurity and information security\n                    - The evolution of cybersecurity practices over time\n                    - The future trends and threats in cybersecurity\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=6419, completion_tokens=99), metadata={}, content=[FunctionCall(id='call_PMT14xeREAb9iA3kTGpoOArE', arguments='{"query": "What is Cybersecurity?"}', name='web_ai_agent'), FunctionCall(id='call_ydsuStRDvxqUEtswOPWEnftT', arguments='{"query": "Differences between cybersecurity a